In [2]:
import pandas as pd
import numpy as np

In [27]:
df1 = pd.DataFrame({'A':list('aba'),'B':[1,3,2]},index=['x',2,3])
df2 = pd.DataFrame({'A':list('aea'),'C':[4,6,5],'B':[1,5,2]},index=list('xyz'))
df1.merge(df2, on='A', how='outer', indicator=True)

,A,B_x,C,B_y,_merge
0,a,1.0,4.0,1.0,both
1,a,1.0,5.0,2.0,both
2,a,2.0,4.0,1.0,both
3,a,2.0,5.0,2.0,both
4,b,3.0,NaN,NaN,left_only
5,e,NaN,6.0,5.0,right_only


In [42]:
df1.assign(a=[1,2])
df1

,Name,Age,Class
0,San Zhang,20,one
1,San Zhang,21,two


In [43]:
df1 = pd.DataFrame({'Name':['San Zhang', 'Si Li', 'Wu Wang'],
                        'Age':[20, 21 ,21],
                        'Class':['one', 'two', 'three']})
df2 = pd.DataFrame({'Name':['San Zhang', 'Li Si', 'Wu Wang'],
                        'Age':[20, 21 ,21],
                        'Class':['one', 'two', 'Three']})
df1.compare(df2)

Name         Class       
    self  other   self  other
1  Si Li  Li Si    NaN    NaN
2    NaN    NaN  three  Three

In [86]:
def choose_min(s1, s2):
    s2 = s2.reindex_like(s1)
    res = s1.where(s1<s2, s2)
    res = res.mask(s2.isna())
    return res
df1 = pd.DataFrame({'A':[1,2], 'B':[3,4], 'C':[5,6]})
df2 = pd.DataFrame({'B':[5,6], 'C':[7,8], 'D':[9,10]}, index=[1,2])
df1.combine(df2, np.minimum, overwrite=False)

,A,B,C,D
0,1.0,NaN,NaN,NaN
1,2.0,4.0,6.0,NaN
2,NaN,NaN,NaN,NaN


In [60]:
ss1 = pd.Series([2,1,0])
ss2 = pd.Series([0,3])
ss2 = ss2.reindex_like(ss1)


0    2.0
1    3.0
2    NaN
dtype: float64

In [57]:
 pd.interval_range(end=5,periods=9,freq=0.5)

IntervalIndex([(0.5, 1.0], (1.0, 1.5], (1.5, 2.0], (2.0, 2.5], (2.5, 3.0], (3.0, 3.5], (3.5, 4.0], (4.0, 4.5], (4.5, 5.0]],
              closed='right',
              dtype='interval[float64]')

In [94]:
df1 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                        'Age':[20, 21],
                        'Class':['one', 'two']})
df2 = pd.DataFrame({'Name':['San Zhang', 'San Zhang'],
                        'Gender':['F', 'M'],
                        'Class':['two', 'one']})
df1.merge(df2, on=['Name', 'Class'], how='left', validate='1:1')

,Name,Age,Class,Gender
0,San Zhang,20,one,M
1,San Zhang,21,two,F


In [83]:
import dataframe_image as dfi
df2.dfi.export('df1.png')